# TRAIN MODEL CNN

In [2]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout, Flatten
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
print(tf.__version__)

2.7.0


In [12]:
images=[]
classNo=[]
path="dataset"

In [13]:
# Load data
for x in range(0, 2):
    myPicList=os.listdir(path+"/"+str(x))
    for y in myPicList:
        curImg=cv2.imread(path+"/"+str(x)+"/"+y)
        curImg=cv2.resize(curImg,(32,32))
        images.append(curImg)
        classNo.append(x)
    print(x)

images=np.array(images)
classNo=np.array(classNo)


0
1


In [21]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test=train_test_split(images, classNo, test_size=0.2)
print(x_train.shape, " ", y_train.shape, " ", y_test.shape, x_test.shape)

(1560, 32, 32, 3)   (1560,)   (390,) (390, 32, 32, 3)


In [22]:
x_train, x_validation, y_train, y_validation=train_test_split(x_train, y_train, test_size=0.2)
print(x_train.shape," ", type(x_train))

(1248, 32, 32, 3)   <class 'numpy.ndarray'>


In [23]:
# cái này sử dụng để normalize ảnh cho việc train
def preprocessing(img):
    img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
   # img=cv2.equalizeHist(img)
    img=img/255 
    return img

In [24]:
# normalize data
x_train= np.array(list(map(preprocessing, x_train)))
x_test= np.array(list(map(preprocessing, x_test)))
x_validation= np.array(list(map(preprocessing, x_validation)))

In [25]:
print(x_train.shape[0], " ", x_train.shape[1], " ", x_train.shape[2])

1248   32   32


In [10]:
#reshape data
x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2],1)
x_test=x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2],1)
x_validation=x_validation.reshape(x_validation.shape[0], x_validation.shape[1], x_validation.shape[2],1)

In [11]:
# tạo constructor ImageDataGenerator 
dataGen=ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.1,
    rotation_range=10)

dataGen.fit(x_train)

# convert interger vector into binary array
y_train=to_categorical(y_train, 2)
y_test=to_categorical(y_test, 2)
y_validation=to_categorical(y_validation, 2)

In [12]:
# train model CNN.
# add convolution layer
# add MaxPolling Layer
# add Dropout Layer
def myModel():
	sizeOfFilter1=(3,3)
	sizeOfFilter2=(3,3)
	sizeOfPool=(2,2)

	model=Sequential()
	model.add((Conv2D(32, (3,3), input_shape=(32,32,1),activation='relu')))
	model.add((Conv2D(32, (3,3),activation='relu')))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add((Conv2D(64, (3,3),activation='relu')))
	model.add((Conv2D(64, (3,3),activation='relu')))
	model.add(MaxPooling2D(pool_size=(2,2)))
	model.add(Dropout(0.5))


	model.add(Flatten())
	model.add(Dense(64, activation='relu'))
	model.add(Dropout(0.5))
	model.add(Dense(2, activation='softmax'))
	model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=['accuracy'])
	return model


In [16]:

model=myModel()

# train
history=model.fit_generator(dataGen.flow(x_train, y_train,batch_size=50),
                        
	epochs=2,
	validation_data=(x_validation,y_validation),
	shuffle=1)


model.save("MyTrainingModel.h5")

Epoch 1/2
25/25 [==============================] - 3s 110ms/step - loss: 0.6874 - accuracy: 0.5521 - val_loss: 0.6627 - val_accuracy: 0.5224
Epoch 2/2
25/25 [==============================] - 2s 99ms/step - loss: 0.6635 - accuracy: 0.5938 - val_loss: 0.5424 - val_accuracy: 0.8558
